In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('movies_metadata.csv',low_memory=False)
df1 = pd.read_csv('credits.csv')
df2 = pd.read_csv('keywords.csv')
df3 = pd.read_csv('links_small.csv')
df4 = pd.read_csv('ratings_small.csv')

In [3]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
df.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [5]:
df.dropna(subset=['imdb_id','runtime','title','release_date'],inplace=True)

In [6]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [7]:
df.isnull().sum()

adult                        0
belongs_to_collection    40639
budget                       0
genres                       0
homepage                 37352
id                           0
imdb_id                      0
original_language           11
original_title               0
overview                   695
popularity                   0
poster_path                320
production_companies         0
production_countries         0
release_date                 0
revenue                      0
runtime                      0
spoken_languages             0
status                      76
tagline                  24722
title                        0
video                        0
vote_average                 0
vote_count                   0
dtype: int64

In [8]:
df['id'] = [int(s) for s in df['id']]
import ast
# s = (df['genres'].iloc[0])
def gen(s):
    ls = ast.literal_eval(s)
    h = ''
    for ele in ls:
        if h == '':
            h = h + ele['name']
        else:
            h = h + ' ' + ele['name']
    return h


In [9]:
df['genres'] = [gen(s) for s in df['genres']]

In [10]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,Animation Comedy Family,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,Adventure Fantasy Family,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,Romance Comedy,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,Comedy Drama Romance,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,Comedy,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [11]:
dff = df[['adult','genres','id','imdb_id','original_title','release_date','title','vote_average','vote_count']]
dff['adult'] = [0 if s == 'False' else 1 for s in dff['adult']]
dff.head()

<ipython-input-11-eed3ae1a07b1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['adult'] = [0 if s == 'False' else 1 for s in dff['adult']]


,adult,genres,id,imdb_id,original_title,release_date,title,vote_average,vote_count
0,0,Animation Comedy Family,862,tt0114709,Toy Story,1995-10-30,Toy Story,7.7,5415.0
1,0,Adventure Fantasy Family,8844,tt0113497,Jumanji,1995-12-15,Jumanji,6.9,2413.0
2,0,Romance Comedy,15602,tt0113228,Grumpier Old Men,1995-12-22,Grumpier Old Men,6.5,92.0
3,0,Comedy Drama Romance,31357,tt0114885,Waiting to Exhale,1995-12-22,Waiting to Exhale,6.1,34.0
4,0,Comedy,11862,tt0113041,Father of the Bride Part II,1995-02-10,Father of the Bride Part II,5.7,173.0


In [12]:
dff['release_date'] = [int(s[:4]) for s in dff['release_date']]


<ipython-input-12-d9ccede11b3d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['release_date'] = [int(s[:4]) for s in dff['release_date']]


In [13]:
df4.rename(columns={'movieId' : 'id'},inplace = True)
df4.head()

,userId,id,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [14]:
df4.shape

(100004, 4)

In [15]:
dff = dff.rename(columns={'release_date':'release_year'})

In [16]:
dff.head()

,adult,genres,id,imdb_id,original_title,release_year,title,vote_average,vote_count
0,0,Animation Comedy Family,862,tt0114709,Toy Story,1995,Toy Story,7.7,5415.0
1,0,Adventure Fantasy Family,8844,tt0113497,Jumanji,1995,Jumanji,6.9,2413.0
2,0,Romance Comedy,15602,tt0113228,Grumpier Old Men,1995,Grumpier Old Men,6.5,92.0
3,0,Comedy Drama Romance,31357,tt0114885,Waiting to Exhale,1995,Waiting to Exhale,6.1,34.0
4,0,Comedy,11862,tt0113041,Father of the Bride Part II,1995,Father of the Bride Part II,5.7,173.0


In [17]:
dff = dff[dff['release_year'] >= 1960]
dff.head()

,adult,genres,id,imdb_id,original_title,release_year,title,vote_average,vote_count
0,0,Animation Comedy Family,862,tt0114709,Toy Story,1995,Toy Story,7.7,5415.0
1,0,Adventure Fantasy Family,8844,tt0113497,Jumanji,1995,Jumanji,6.9,2413.0
2,0,Romance Comedy,15602,tt0113228,Grumpier Old Men,1995,Grumpier Old Men,6.5,92.0
3,0,Comedy Drama Romance,31357,tt0114885,Waiting to Exhale,1995,Waiting to Exhale,6.1,34.0
4,0,Comedy,11862,tt0113041,Father of the Bride Part II,1995,Father of the Bride Part II,5.7,173.0


In [18]:
dff.shape

(39465, 9)

In [19]:
df1.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [20]:
dff = dff.merge(df1,on='id')
dff.head()

,adult,genres,id,imdb_id,original_title,release_year,title,vote_average,vote_count,cast,crew
0,0,Animation Comedy Family,862,tt0114709,Toy Story,1995,Toy Story,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,0,Adventure Fantasy Family,8844,tt0113497,Jumanji,1995,Jumanji,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,0,Romance Comedy,15602,tt0113228,Grumpier Old Men,1995,Grumpier Old Men,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,0,Comedy Drama Romance,31357,tt0114885,Waiting to Exhale,1995,Waiting to Exhale,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,0,Comedy,11862,tt0113041,Father of the Bride Part II,1995,Father of the Bride Part II,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


In [21]:
def director(s):
    ls = ast.literal_eval(s)
    for i in ls:
        if i['job'] == 'Director':
            return i['name']
    return None

In [22]:
dff['Director'] = [director(s) for s in dff['crew']]

In [23]:
dff.head()

,adult,genres,id,imdb_id,original_title,release_year,title,vote_average,vote_count,cast,crew,Director
0,0,Animation Comedy Family,862,tt0114709,Toy Story,1995,Toy Story,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",John Lasseter
1,0,Adventure Fantasy Family,8844,tt0113497,Jumanji,1995,Jumanji,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",Joe Johnston
2,0,Romance Comedy,15602,tt0113228,Grumpier Old Men,1995,Grumpier Old Men,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",Howard Deutch
3,0,Comedy Drama Romance,31357,tt0114885,Waiting to Exhale,1995,Waiting to Exhale,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",Forest Whitaker
4,0,Comedy,11862,tt0113041,Father of the Bride Part II,1995,Father of the Bride Part II,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",Charles Shyer


In [24]:
def allCast(s):
    ls = ast.literal_eval(s)
#     return ls
    h = ''
    for i in ls:
        p = i['name']
        if h == '': 
            h = h + p
        else:
            h = h + ' ' + p
    return h

In [25]:
dff['cast'] = [allCast(s) for s in dff['cast']]

In [26]:
dff.head()

,adult,genres,id,imdb_id,original_title,release_year,title,vote_average,vote_count,cast,crew,Director
0,0,Animation Comedy Family,862,tt0114709,Toy Story,1995,Toy Story,7.7,5415.0,Tom Hanks Tim Allen Don Rickles Jim Varney Wal...,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",John Lasseter
1,0,Adventure Fantasy Family,8844,tt0113497,Jumanji,1995,Jumanji,6.9,2413.0,Robin Williams Jonathan Hyde Kirsten Dunst Bra...,"[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",Joe Johnston
2,0,Romance Comedy,15602,tt0113228,Grumpier Old Men,1995,Grumpier Old Men,6.5,92.0,Walter Matthau Jack Lemmon Ann-Margret Sophia ...,"[{'credit_id': '52fe466a9251416c75077a89', 'de...",Howard Deutch
3,0,Comedy Drama Romance,31357,tt0114885,Waiting to Exhale,1995,Waiting to Exhale,6.1,34.0,Whitney Houston Angela Bassett Loretta Devine ...,"[{'credit_id': '52fe44779251416c91011acb', 'de...",Forest Whitaker
4,0,Comedy,11862,tt0113041,Father of the Bride Part II,1995,Father of the Bride Part II,5.7,173.0,Steve Martin Diane Keaton Martin Short Kimberl...,"[{'credit_id': '52fe44959251416c75039ed7', 'de...",Charles Shyer


In [27]:
dff_mod = dff
dff_mod = dff_mod.drop(['crew'],axis=1)
dff_mod.head()

,adult,genres,id,imdb_id,original_title,release_year,title,vote_average,vote_count,cast,Director
0,0,Animation Comedy Family,862,tt0114709,Toy Story,1995,Toy Story,7.7,5415.0,Tom Hanks Tim Allen Don Rickles Jim Varney Wal...,John Lasseter
1,0,Adventure Fantasy Family,8844,tt0113497,Jumanji,1995,Jumanji,6.9,2413.0,Robin Williams Jonathan Hyde Kirsten Dunst Bra...,Joe Johnston
2,0,Romance Comedy,15602,tt0113228,Grumpier Old Men,1995,Grumpier Old Men,6.5,92.0,Walter Matthau Jack Lemmon Ann-Margret Sophia ...,Howard Deutch
3,0,Comedy Drama Romance,31357,tt0114885,Waiting to Exhale,1995,Waiting to Exhale,6.1,34.0,Whitney Houston Angela Bassett Loretta Devine ...,Forest Whitaker
4,0,Comedy,11862,tt0113041,Father of the Bride Part II,1995,Father of the Bride Part II,5.7,173.0,Steve Martin Diane Keaton Martin Short Kimberl...,Charles Shyer


In [28]:
df4.head()

,userId,id,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [29]:
df4.shape
df4_mod = df4.drop(['timestamp'],axis = 1)
df4_mod.head()

,userId,id,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [30]:
from sklearn.model_selection import train_test_split
train_df,test_df = train_test_split(df4_mod,test_size = 0.3,random_state = 1)

In [31]:
train_df.head()
train_df.shape

(70002, 3)

In [32]:
test_df.shape

(30002, 3)

In [33]:
from surprise import SVD
import numpy as np
import surprise
from surprise import Reader, Dataset

reader = Reader(rating_scale=(1,5))
reader

train_data_mf = Dataset.load_from_df(train_df[['userId', 'id', 'rating']], reader)
trainset = train_data_mf.build_full_trainset()

svd = SVD(n_factors=100, biased=True, random_state=15, verbose=True)
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [34]:
train_preds = svd.test(trainset.build_testset())
train_pred_mf = np.array([pred.est for pred in train_preds])

In [35]:
test_data_mf = Dataset.load_from_df(test_df[['userId', 'id', 'rating']], reader)
testset = test_data_mf.build_full_trainset()

svd = SVD(n_factors=100, biased=True, random_state=15, verbose=True)
svd.fit(testset)

test_preds = svd.test(testset.build_testset())
test_pred_mf = np.array([pred.est for pred in test_preds])

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [36]:
# Creating a sparse matrix
from scipy.sparse import csr_matrix
train_sparse_matrix = csr_matrix((train_df.rating.values, (train_df.userId.values, train_df.id.values)))

In [37]:
from scipy.sparse import csr_matrix
test_sparse_matrix = csr_matrix((test_df.rating.values, (test_df.userId.values, test_df.id.values)))

In [38]:
train_averages = dict()
train_global_average = train_sparse_matrix.sum()/train_sparse_matrix.count_nonzero()
train_averages['global'] = train_global_average
train_averages

{'global': 3.543363046770092}

In [39]:
test_averages = dict()
test_global_average = test_sparse_matrix.sum()/test_sparse_matrix.count_nonzero()
test_averages['global'] = test_global_average
test_averages

{'global': 3.544180387974135}

In [40]:
def get_average_ratings(sparse_matrix, of_users):
    ax = 1 if of_users else 0 
    sum_of_ratings = sparse_matrix.sum(axis=ax).A1
    is_rated = sparse_matrix!=0
    no_of_ratings = is_rated.sum(axis=ax).A1
    u,m = sparse_matrix.shape
    average_ratings = { i : sum_of_ratings[i]/no_of_ratings[i]for i in range(u if of_users else m) if no_of_ratings[i] !=0}
    return average_ratings

In [41]:
train_averages['user'] = get_average_ratings(train_sparse_matrix, of_users=True)
train_averages['movie'] = get_average_ratings(train_sparse_matrix, of_users=False)

In [42]:
test_averages['user'] = get_average_ratings(test_sparse_matrix, of_users=True)
test_averages['movie'] = get_average_ratings(test_sparse_matrix, of_users=False)

In [43]:
test_averages['user'][1]

3.0714285714285716

In [44]:
train_df.head()

,userId,id,rating
13499,87,733,3.0
90426,599,50954,4.0
23841,168,608,5.0
24559,177,338,4.0
7599,48,46578,3.5


In [45]:
train_df['userAvg'] = [train_averages['user'][s] for s in train_df['userId']]

<ipython-input-45-5272c5960c91>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['userAvg'] = [train_averages['user'][s] for s in train_df['userId']]


In [46]:
test_df['userAvg'] = [test_averages['user'][s] for s in test_df['userId']]

<ipython-input-46-d9bdc401d13c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['userAvg'] = [test_averages['user'][s] for s in test_df['userId']]


In [47]:
train_df.head()

,userId,id,rating,userAvg
13499,87,733,3.0,3.200000
90426,599,50954,4.0,3.969925
23841,168,608,5.0,3.341463
24559,177,338,4.0,3.913580
7599,48,46578,3.5,3.502653


In [48]:
train_df['movieAvg'] = [train_averages['movie'][s] for s in train_df['id']]

<ipython-input-48-cfab38c10338>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['movieAvg'] = [train_averages['movie'][s] for s in train_df['id']]


In [49]:
test_df['movieAvg'] = [test_averages['movie'][s] for s in test_df['id']]

<ipython-input-49-6a8d49cdde97>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['movieAvg'] = [test_averages['movie'][s] for s in test_df['id']]


In [50]:
train_df.head()

,userId,id,rating,userAvg,movieAvg
13499,87,733,3.0,3.200000,3.739796
90426,599,50954,4.0,3.969925,4.000000
23841,168,608,5.0,3.341463,4.257576
24559,177,338,4.0,3.913580,2.875000
7599,48,46578,3.5,3.502653,3.800000


In [51]:
def simUsers(user,movie):
    user_sim = cosine_similarity(train_sparse_matrix[user], train_sparse_matrix).ravel()
    top_sim_users = user_sim.argsort()[::-1][1:] 
    top_ratings = train_sparse_matrix[top_sim_users, movie].toarray().ravel()
    top_sim_users_ratings = list(top_ratings[top_ratings != 0][:5])
    top_sim_users_ratings.extend([train_averages['movie'][movie]]*(5 -len(top_sim_users_ratings)))
    return top_sim_users_ratings
    

In [52]:
def simUsers(user,movie):
    user_sim = cosine_similarity(test_sparse_matrix[user], test_sparse_matrix).ravel()
    top_sim_users = user_sim.argsort()[::-1][1:] 
    top_ratings = test_sparse_matrix[top_sim_users, movie].toarray().ravel()
    top_sim_users_ratings = list(top_ratings[top_ratings != 0][:5])
    top_sim_users_ratings.extend([test_averages['movie'][movie]]*(5 -len(top_sim_users_ratings)))
    return top_sim_users_ratings

In [53]:
h_df = train_df[['userId','id']]
h_df.head()
r1,c1 = h_df.shape
r1

70002

In [54]:
h1_df = test_df[['userId','id']]
h1_df.head()
r2,c2 = h1_df.shape
r2

30002

In [55]:
user = h_df.iloc[0]
user

userId     87
id        733
Name: 13499, dtype: int64

In [56]:
# out = []
# for i in range(len(h_df)):
#     p = h_df.iloc[i]
#     user = p['userId']
#     movie = p['id']
#     l = simUsers(user,movie)
#     l.append(user)
#     l.append(movie)
#     out.append(l)

In [57]:
# out1 = []
# for i in range(len(h1_df)):
#     p1 = h1_df.iloc[i]
#     user = p1['userId']
#     movie = p1['id']
#     lt = simUsers(user,movie)
#     lt.append(user)
#     lt.append(movie)
#     out1.append(lt)

In [58]:
# print(out)

In [59]:
# print(out1)

In [60]:
# user_df = pd.DataFrame(out, columns =['simU1', 'simU2', 'simU3','simU4','simU5','userId','id'])
# user_df.head()

In [61]:
# usert_df = pd.DataFrame(out1, columns =['simU1', 'simU2', 'simU3','simU4','simU5','userId','id'])
# usert_df.head()

In [62]:
# train_df = pd.merge(train_df, user_df,  how='left', left_on=['userId','id'], right_on = ['userId','id'])

In [63]:
# test_df = pd.merge(test_df, usert_df,  how='left', left_on=['userId','id'], right_on = ['userId','id'])
# test_df.head()

In [64]:
train_df.head()

,userId,id,rating,userAvg,movieAvg
13499,87,733,3.0,3.200000,3.739796
90426,599,50954,4.0,3.969925,4.000000
23841,168,608,5.0,3.341463,4.257576
24559,177,338,4.0,3.913580,2.875000
7599,48,46578,3.5,3.502653,3.800000


In [65]:
def simMovies(user,movie):
    movie_sim = cosine_similarity(train_sparse_matrix[:,movie].T,
    train_sparse_matrix.T).ravel()
    top_sim_movies = movie_sim.argsort()[::-1][1:]
    top_ratings = train_sparse_matrix[user, top_sim_movies].toarray().ravel()
    top_sim_movies_ratings = list(top_ratings[top_ratings != 0][:5])
    top_sim_movies_ratings.extend([train_averages['user'][user]]*(5-len(top_sim_movies_ratings)))
    return top_sim_movies_ratings

In [66]:
def simMovies(user,movie):
    movie_sim = cosine_similarity(test_sparse_matrix[:,movie].T,
    test_sparse_matrix.T).ravel()
    top_sim_movies = movie_sim.argsort()[::-1][1:]
    top_ratings = test_sparse_matrix[user, top_sim_movies].toarray().ravel()
    top_sim_movies_ratings = list(top_ratings[top_ratings != 0][:5])
    top_sim_movies_ratings.extend([test_averages['user'][user]]*(5-len(top_sim_movies_ratings)))
    return top_sim_movies_ratings

In [67]:
# h_df.head()
# movie = h_df.iloc[0]
# movie
# out1 = []
# for i in range(len(h_df)):
#     p = h_df.iloc[i]
#     user = p['userId']
#     movie = p['id']
#     l1 = simMovies(user,movie)
#     l1.append(user)
#     l1.append(movie)
#     out1.append(l1)

In [68]:
# h1_df.head()
# movie = h1_df.iloc[0]
# movie
# out3 = []
# for i in range(len(h1_df)):
#     p3 = h1_df.iloc[i]
#     user = p3['userId']
#     movie = p3['id']
#     l3 = simMovies(user,movie)
#     l3.append(user)
#     l3.append(movie)
#     out3.append(l3)

In [69]:
# print(out3)

In [70]:
# print(out1)

In [71]:
# simM_df = pd.DataFrame(out, columns =['simM1', 'simM2', 'simM3','simM4','simM5','userId','id'])
# simM_df.head()

In [72]:
# simMt_df = pd.DataFrame(out3, columns =['simM1', 'simM2', 'simM3','simM4','simM5','userId','id'])
# simMt_df.head()

In [73]:
# train_df = pd.merge(train_df, simM_df,  how='left', left_on=['userId','id'], right_on = ['userId','id'])
# train_df.head()

In [74]:
# test_df = pd.merge(test_df, simMt_df,  how='left', left_on=['userId','id'], right_on = ['userId','id'])
# test_df.head()

In [75]:
# train_df.to_csv('features.csv')

In [76]:
# test_df.to_csv('features_test.csv')

In [77]:
t_df = pd.read_csv('features.csv')

t_df.head()


,Unnamed: 0,userId,id,rating,userAvg,movieAvg,simU1,simU2,simU3,simU4,simU5,simM1,simM2,simM3,simM4,simM5
0,0,87,733,3.0,3.200000,3.739796,4.0,3.0,5.0,4.0,4.0,4.0,3.0,5.0,4.0,4.0
1,1,599,50954,4.0,3.969925,4.000000,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
2,2,168,608,5.0,3.341463,4.257576,5.0,4.0,4.0,5.0,4.0,5.0,4.0,4.0,5.0,4.0
3,3,177,338,4.0,3.913580,2.875000,4.0,2.0,3.0,3.0,2.0,4.0,2.0,3.0,3.0,2.0
4,4,48,46578,3.5,3.502653,3.800000,3.0,4.5,4.0,3.5,4.5,3.0,4.5,4.0,3.5,4.5


In [78]:
t1_df = pd.read_csv('features_test.csv')

t1_df.head()

,Unnamed: 0,userId,id,rating,userAvg,movieAvg,simU1,simU2,simU3,simU4,simU5,simM1,simM2,simM3,simM4,simM5
0,0,667,446,4.0,3.750000,3.750000,3.0,3.0,5.0,3.75,3.75,3.0,5.0,5.0,3.0,5.0
1,1,104,71135,3.5,3.880000,4.000000,4.0,4.5,4.0,4.00,4.00,3.5,3.5,4.0,4.0,3.5
2,2,19,1394,5.0,3.409449,4.117647,5.0,5.0,4.0,4.00,5.00,5.0,3.0,5.0,4.0,4.0
3,3,395,671,5.0,3.400000,4.291667,4.0,5.0,4.0,4.00,4.00,3.0,2.0,4.0,2.0,3.0
4,4,187,953,3.0,3.785714,4.000000,5.0,4.5,5.0,5.00,4.00,3.5,2.5,3.0,4.0,0.5


In [79]:
t_df.head()

,Unnamed: 0,userId,id,rating,userAvg,movieAvg,simU1,simU2,simU3,simU4,simU5,simM1,simM2,simM3,simM4,simM5
0,0,87,733,3.0,3.200000,3.739796,4.0,3.0,5.0,4.0,4.0,4.0,3.0,5.0,4.0,4.0
1,1,599,50954,4.0,3.969925,4.000000,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
2,2,168,608,5.0,3.341463,4.257576,5.0,4.0,4.0,5.0,4.0,5.0,4.0,4.0,5.0,4.0
3,3,177,338,4.0,3.913580,2.875000,4.0,2.0,3.0,3.0,2.0,4.0,2.0,3.0,3.0,2.0
4,4,48,46578,3.5,3.502653,3.800000,3.0,4.5,4.0,3.5,4.5,3.0,4.5,4.0,3.5,4.5


In [80]:
t_df = t_df.drop(['Unnamed: 0'],axis = 1)

In [81]:
t1_df = t1_df.drop(['Unnamed: 0'],axis = 1)

In [82]:
s_df = pd.DataFrame(train_pred_mf,columns=['mf_svd'])
t_df = pd.concat([t_df,s_df],axis=1)
t_df.head()

,userId,id,rating,userAvg,movieAvg,simU1,simU2,simU3,simU4,simU5,simM1,simM2,simM3,simM4,simM5,mf_svd
0,87,733,3.0,3.200000,3.739796,4.0,3.0,5.0,4.0,4.0,4.0,3.0,5.0,4.0,4.0,3.173744
1,599,50954,4.0,3.969925,4.000000,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,2.616064
2,168,608,5.0,3.341463,4.257576,5.0,4.0,4.0,5.0,4.0,5.0,4.0,4.0,5.0,4.0,3.336979
3,177,338,4.0,3.913580,2.875000,4.0,2.0,3.0,3.0,2.0,4.0,2.0,3.0,3.0,2.0,3.468330
4,48,46578,3.5,3.502653,3.800000,3.0,4.5,4.0,3.5,4.5,3.0,4.5,4.0,3.5,4.5,3.008532


In [83]:
s1_df = pd.DataFrame(test_pred_mf,columns=['mf_svd'])
t1_df = pd.concat([t1_df,s1_df],axis=1)
t1_df.head()

,userId,id,rating,userAvg,movieAvg,simU1,simU2,simU3,simU4,simU5,simM1,simM2,simM3,simM4,simM5,mf_svd
0,667,446,4.0,3.750000,3.750000,3.0,3.0,5.0,3.75,3.75,3.0,5.0,5.0,3.0,5.0,3.909319
1,104,71135,3.5,3.880000,4.000000,4.0,4.5,4.0,4.00,4.00,3.5,3.5,4.0,4.0,3.5,3.902243
2,19,1394,5.0,3.409449,4.117647,5.0,5.0,4.0,4.00,5.00,5.0,3.0,5.0,4.0,4.0,3.778042
3,395,671,5.0,3.400000,4.291667,4.0,5.0,4.0,4.00,4.00,3.0,2.0,4.0,2.0,3.0,3.703699
4,187,953,3.0,3.785714,4.000000,5.0,4.5,5.0,5.00,4.00,3.5,2.5,3.0,4.0,0.5,3.483841


In [84]:
final_df = t_df
t_df.head()

,userId,id,rating,userAvg,movieAvg,simU1,simU2,simU3,simU4,simU5,simM1,simM2,simM3,simM4,simM5,mf_svd
0,87,733,3.0,3.200000,3.739796,4.0,3.0,5.0,4.0,4.0,4.0,3.0,5.0,4.0,4.0,3.173744
1,599,50954,4.0,3.969925,4.000000,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,2.616064
2,168,608,5.0,3.341463,4.257576,5.0,4.0,4.0,5.0,4.0,5.0,4.0,4.0,5.0,4.0,3.336979
3,177,338,4.0,3.913580,2.875000,4.0,2.0,3.0,3.0,2.0,4.0,2.0,3.0,3.0,2.0,3.468330
4,48,46578,3.5,3.502653,3.800000,3.0,4.5,4.0,3.5,4.5,3.0,4.5,4.0,3.5,4.5,3.008532


In [85]:
final_test_df = t1_df

In [86]:
import sys
!{sys.executable} -m pip install xgboost

In [87]:
# prepare train data
x_train = final_df.drop(['userId','id','rating'], axis=1)
y_train = final_df['rating']

#XGB
import xgboost as xgb

xgb_model = xgb.XGBRegressor(silent=False, n_jobs=13,random_state=15,n_estimators=100)
# fit the model
xgb_model.fit(x_train, y_train, eval_metric = 'rmse')

[10:41:19] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-030221e36e1a46bfb-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "silent" } are not used.


C:\Users\DELL\anaconda3\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=13,
             num_parallel_tree=1, predictor='auto', random_state=15, ...)

In [88]:
y_train_pred = xgb_model.predict(x_train)
rmse = np.sqrt(np.mean([ (y_train.values[i] - y_train_pred[i])**2 for i in range(len(y_train_pred)) ]))
mape = np.mean(np.abs( (y_train.values- y_train_pred)/y_train.values )) * 100
# store the results in train_results dictionary
train_results = {'rmse': rmse,'mape' : mape,'predictions' : y_train_pred}

In [89]:
print(train_results)

{'rmse': 0.7382854386697496, 'mape': 22.828892827680086, 'predictions': array([3.3051076, 4.2043953, 4.090516 , ..., 4.549897 , 4.0599065,
       4.3136835], dtype=float32)}


In [90]:
#dictionaries for storing train and test results
test_results = dict()
x_test = final_test_df.drop(['userId','id','rating'], axis=1)
y_test = final_test_df['rating']
# from the trained model, get the predictions
y_test_pred = xgb_model.predict(x_test)
rmse = np.sqrt(np.mean([ (y_test.values[i] - y_test_pred[i])**2 for i in range(len(y_test_pred)) ]))
mape = np.mean(np.abs( (y_test.values- y_test_pred)/y_test.values )) * 100
# store the results in train_results dictionary
test_results = {'rmse': rmse,'mape' : mape,'predictions' : y_test_pred}

In [91]:
print(test_results)

{'rmse': 0.7617246644703336, 'mape': 22.711990869521543, 'predictions': array([4.046579 , 4.0642753, 3.9808505, ..., 3.9882205, 2.675646 ,
       3.9303248], dtype=float32)}


In [92]:
final_test_df['predictions'] = [s for s in y_test_pred]
final_df['predictions'] = [s for s in y_train_pred]

In [93]:
final_test_df = t1_df
final_test_df.head()
final_df.to_csv('train_movie_data.csv')
final_test_df.to_csv('test_movie_data.csv')

In [94]:
train = pd.read_csv('train_movie_data.csv')
test = pd.read_csv('test_movie_data.csv')
final_mod_df = pd.concat([train,test])

In [95]:
final_mod_df

,Unnamed: 0,userId,id,rating,userAvg,movieAvg,simU1,simU2,simU3,simU4,simU5,simM1,simM2,simM3,simM4,simM5,mf_svd,predictions
0,0,87,733,3.0,3.200000,3.739796,4.0,3.00,5.00,4.00,4.00,4.0,3.0,5.0,4.0,4.0,3.173744,3.305108
1,1,599,50954,4.0,3.969925,4.000000,4.0,4.00,4.00,4.00,4.00,4.0,4.0,4.0,4.0,4.0,2.616064,4.204395
2,2,168,608,5.0,3.341463,4.257576,5.0,4.00,4.00,5.00,4.00,5.0,4.0,4.0,5.0,4.0,3.336979,4.090516
3,3,177,338,4.0,3.913580,2.875000,4.0,2.00,3.00,3.00,2.00,4.0,2.0,3.0,3.0,2.0,3.468330,3.238984
4,4,48,46578,3.5,3.502653,3.800000,3.0,4.50,4.00,3.50,4.50,3.0,4.5,4.0,3.5,4.5,3.008532,3.728580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29997,29997,547,45722,3.5,3.398836,3.315789,2.0,2.50,4.00,2.50,5.00,4.0,5.0,4.5,4.5,4.0,3.704031,3.047854
29998,29998,73,1595,0.5,3.353061,0.750000,1.0,0.75,0.75,0.75,0.75,2.5,0.5,4.5,3.5,4.5,3.924975,0.787984
29999,29999,486,2324,4.0,3.597826,3.821429,3.5,3.50,4.50,2.50,4.00,5.0,5.0,2.5,5.0,4.0,3.562906,3.988220
30000,30000,133,593,0.5,2.084746,4.066372,4.0,4.50,2.00,3.50,3.00,3.5,0.5,4.0,1.0,4.0,4.053895,2.675646


In [96]:
final_mod_df = final_mod_df.sort_values(['id','userId'],ascending =[True,True])

In [97]:
final_mod_df.head()

,Unnamed: 0,userId,id,rating,userAvg,movieAvg,simU1,simU2,simU3,simU4,simU5,simM1,simM2,simM3,simM4,simM5,mf_svd,predictions
24209,24209,7,1,3.0,3.400000,3.869048,5.0,4.0,4.0,5.0,3.0,3.0,1.0,3.0,4.0,4.0,2.992632,3.461638
25378,25378,9,1,4.0,3.676471,3.874233,3.0,4.0,4.0,4.5,5.0,3.0,4.0,4.0,4.5,5.0,3.471999,3.934644
53815,53815,13,1,5.0,3.675000,3.874233,5.0,4.0,4.5,3.5,3.0,5.0,4.0,4.5,3.5,3.0,3.730859,3.936111
36825,36825,15,1,2.0,2.626054,3.874233,3.0,4.0,4.0,1.5,4.0,3.0,4.0,4.0,1.5,4.0,3.767999,3.020684
4086,4086,19,1,3.0,3.409449,3.869048,3.0,3.5,5.0,4.0,4.0,3.0,3.0,3.0,3.0,4.0,3.873037,3.486386


In [98]:
final_mod_df.shape

(100004, 18)